## Converting R code to Python

This notebook aims to answer number 2 of the assignment:

2. Convert the R codes into Python Codes (use jupyter notebook).

This is a direct translation of SEE.R file to python

To look at the comparison between SEE-K-means and SEE-(Another Clustering), head to [main.ipynb](./main.ipynb)

### Import Libraries

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

### Define Function/s

In [13]:
def See(arg1, tidy):
    # Filter rows where ATC equals arg1
    C09CA01 = tidy[tidy['ATC'] == arg1].copy()

    # Make working copies
    Drug_see_p0 = C09CA01.copy()
    Drug_see_p1 = C09CA01.copy()

    # Sort by 'pnr' and 'eksd' and create a lag of 'eksd' by group 'pnr'
    Drug_see_p1.sort_values(by=['pnr', 'eksd'], inplace=True)
    Drug_see_p1['prev_eksd'] = Drug_see_p1.groupby('pnr')['eksd'].shift(1)
    Drug_see_p1 = Drug_see_p1.dropna(subset=['prev_eksd'])

    # For each pnr group, randomly sample one row
    Drug_see_p1 = Drug_see_p1.groupby('pnr', group_keys=False).apply(lambda x: x.sample(1))

    # Keep only needed columns
    Drug_see_p1 = Drug_see_p1[['pnr', 'eksd', 'prev_eksd']].copy()

    # Compute the event interval (if eksd is datetime, compute days; otherwise, assume numeric)
    if np.issubdtype(Drug_see_p1['eksd'].dtype, np.datetime64):
        Drug_see_p1['event.interval'] = (Drug_see_p1['eksd'] - Drug_see_p1['prev_eksd']).dt.days
    else:
        Drug_see_p1['event.interval'] = Drug_see_p1['eksd'] - Drug_see_p1['prev_eksd']
    Drug_see_p1['event.interval'] = pd.to_numeric(Drug_see_p1['event.interval'])

    # --- ECDF Generation ---
    # Compute the empirical CDF for event.interval
    sorted_intervals = np.sort(Drug_see_p1['event.interval'].values)
    n = len(sorted_intervals)
    yvals = np.arange(1, n+1) / n
    dfper = pd.DataFrame({'x': sorted_intervals, 'y': yvals})

    # Retain the lower 80% of the ECDF
    dfper_80 = dfper[dfper['y'] <= 0.8].copy()
    ni = dfper_80['x'].max()  # maximum event interval in the 80% subset

    # Plot ECDFs
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].scatter(dfper_80['x'], dfper_80['y'])
    axs[0].set_title("80% ECDF")
    axs[0].set_xlabel("event.interval")
    axs[0].set_ylabel("ECDF")
    axs[1].scatter(dfper['x'], dfper['y'])
    axs[1].set_title("100% ECDF")
    axs[1].set_xlabel("event.interval")
    axs[1].set_ylabel("ECDF")
    plt.tight_layout()
    plt.show()

    # Plot frequency table for pnr counts
    m1 = Drug_see_p1['pnr'].value_counts()
    m1.plot(kind='bar', title="Frequency of pnr")
    plt.xlabel("pnr")
    plt.ylabel("Count")
    plt.show()

    # Subset to rows with event.interval <= ni (using the same dataframe as before)
    Drug_see_p2 = Drug_see_p1[Drug_see_p1['event.interval'] <= ni].copy()

    # --- Density Estimation on Log(event.interval) ---
    log_intervals = np.log(Drug_see_p2['event.interval'].values)
    kde = gaussian_kde(log_intervals)
    x1 = np.linspace(log_intervals.min(), log_intervals.max(), 100)
    y1 = kde(x1)

    plt.plot(x1, y1)
    plt.title("Density of log(event.interval)")
    plt.xlabel("log(event.interval)")
    plt.ylabel("Density")
    plt.show()

    # Prepare data for silhouette analysis: scale the density grid points
    a = pd.DataFrame({'x': x1, 'y': y1})
    scaler = StandardScaler()
    a_scaled = scaler.fit_transform(a)

    # --- Silhouette Analysis to determine optimal number of clusters ---
    # Try clustering with k from 2 to 10 (or up to the number of points)
    best_k = 2
    best_score = -1
    max_k = min(10, len(a_scaled))
    for k in range(2, max_k + 1):
        km = KMeans(n_clusters=k, random_state=1234)
        labels = km.fit_predict(a_scaled)
        score = silhouette_score(a_scaled, labels)
        if score > best_score:
            best_score = score
            best_k = k
    max_cluster = best_k

    # --- K-means Clustering on the ECDF x-values ---
    # Reshape x for clustering (one-dimensional)
    kmeans = KMeans(n_clusters=max_cluster, random_state=1234)
    dfper['cluster'] = kmeans.fit_predict(dfper[['x']])

    # Compute summary statistics (min, max, median) on log(x) for each cluster
    dfper['log_x'] = np.log(dfper['x'])
    summary = dfper.groupby('cluster')['log_x'].agg(['min', 'max', 'median']).reset_index()
    # Replace any -infinity with 0 if necessary
    summary['min'] = summary['min'].replace(-np.inf, 0)
    # Exponentiate to return to original scale
    summary['Minimum'] = np.exp(summary['min'])
    summary['Maximum'] = np.exp(summary['max'])
    summary['Median'] = np.exp(summary['median'])
    # Keep clusters with a positive median
    summary = summary[summary['Median'] > 0]

    # --- Cross-join with Drug_see_p1 and assign a cluster if event.interval falls within the cluster bounds ---
    Drug_see_p1['key'] = 1
    summary['key'] = 1
    cross = pd.merge(Drug_see_p1, summary, on='key')
    cross['Final_cluster'] = np.where(
        (cross['event.interval'] >= cross['Minimum']) & (cross['event.interval'] <= cross['Maximum']),
        cross['cluster'],
        np.nan
    )
    results = cross.dropna(subset=['Final_cluster']).copy()
    # Keep only the needed columns (using the cluster from ECDF, not Final_cluster, as in the R code)
    results = results[['pnr', 'Median', 'cluster']]

    # --- Choose the cluster with the highest frequency ---
    if not results.empty:
        top_cluster = results['cluster'].value_counts().idxmax()
        t1 = results[results['cluster'] == top_cluster].iloc[[0]].copy()
    else:
        t1 = pd.DataFrame({'cluster': [0], 'Median': [np.nan]})

    # --- Merge cluster results back into Drug_see_p1 ---
    Drug_see_p1 = pd.merge(Drug_see_p1, results, on='pnr', how='left', suffixes=('', '_res'))
    # Fill missing Median values with the top cluster’s median and missing clusters with 0
    top_median = t1['Median'].iloc[0] if not t1.empty else np.nan
    Drug_see_p1['Median'] = Drug_see_p1['Median'].fillna(top_median)
    Drug_see_p1['cluster'] = Drug_see_p1['cluster'].fillna(0)
    # Compute a test column as the difference between event.interval and Median (rounded to one decimal)
    Drug_see_p1['test'] = np.round(Drug_see_p1['event.interval'] - Drug_see_p1['Median'], 1)

    # Keep only pnr, Median, and cluster in a separate dataframe
    Drug_see_p3 = Drug_see_p1[['pnr', 'Median', 'cluster']].copy()

    # --- Merge back with the original copy (Drug_see_p0) ---
    Drug_see_p0 = pd.merge(Drug_see_p0, Drug_see_p3, on='pnr', how='left')
    Drug_see_p0['Median'] = Drug_see_p0['Median'].fillna(top_median)
    Drug_see_p0['cluster'] = Drug_see_p0['cluster'].fillna(0)

    return Drug_see_p0


In [14]:
def see_assumption(arg1):
    # Ensure data is sorted by 'pnr' and 'eksd'
    arg1 = arg1.sort_values(by=['pnr', 'eksd']).copy()

    # Create lag column 'prev_eksd' for each patient group
    arg1['prev_eksd'] = arg1.groupby('pnr')['eksd'].shift(1)

    # Create a sequential prescription number for each patient (starting at 1)
    arg1['p_number'] = arg1.groupby('pnr').cumcount() + 1

    # Filter to keep only rows where p_number is 2 or higher
    Drug_see2 = arg1[arg1['p_number'] >= 2].copy()

    # Select only the needed columns
    Drug_see2 = Drug_see2[['pnr', 'eksd', 'prev_eksd', 'p_number']]

    # Calculate Duration as the difference in days between eksd and prev_eksd
    Drug_see2['Duration'] = (Drug_see2['eksd'] - Drug_see2['prev_eksd']).dt.days

    # Convert p_number to a categorical type (like a factor in R)
    Drug_see2['p_number'] = Drug_see2['p_number'].astype('category')

    # Create a boxplot of Duration versus p_number
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.boxplot(x='p_number', y='Duration', data=Drug_see2, ax=ax)
    ax.set_title("Boxplot of Duration by p_number")

    # Compute the median duration for each patient (pnr)
    medians_of_medians = (
        Drug_see2.groupby('pnr')['Duration']
        .median()
        .reset_index(name='median_duration')
    )

    # Plot a horizontal dashed red line for each patient's median duration
    # (Note: This may add many lines if there are many patients.)
    for med in medians_of_medians['median_duration']:
        ax.axhline(y=med, color='red', linestyle='dashed', alpha=0.5)

    plt.tight_layout()
    return fig